In [1]:
import jijmodeling as jm


def build_drr_model_exact():
    P = jm.Problem("2RR Scheduling")

    # -------------------------
    # Sizes (scalars) = ZPL params and set sizes
    # -------------------------
    T = jm.Placeholder("T")  # |T| (ZPL: param teams)
    S = jm.Placeholder("S")  # |S| (ZPL: param slots)
    S0 = jm.Placeholder("S0")  # |S0| = S-1 (ZPL: S0 := {1..S-1})
    S1 = jm.Placeholder("S1")  # |S1| = S/2 (ZPL: S1 := {0..S/2-1})
    S2 = jm.Placeholder("S2")  # |S2| = S/2 (ZPL: S2 := {S/2..S-1})
    SZ = jm.Placeholder("SZ")  # |SZ| = S-1 (ZPL: SZ := {0..S-2})
    matches_per_slot = jm.Placeholder(
        "matches_per_slot"
    )  # ZPL: param matches_per_slot = |T|/2

    # -------------------------
    # Explicit index lists for sets (mirror ZPL set literals)
    # -------------------------
    T_idx = jm.Placeholder("T_idx", ndim=1)  # [0..T-1]
    S_idx = jm.Placeholder("S_idx", ndim=1)  # [0..S-1]
    S0_idx = jm.Placeholder("S0_idx", ndim=1)  # [1..S-1]  (used by br_count and br1)
    S1_idx = jm.Placeholder("S1_idx", ndim=1)  # [0..S/2 - 1] (used by c4)
    S2_idx = jm.Placeholder(
        "S2_idx", ndim=1
    )  # [S/2..S-1]    (not strictly needed but kept for completeness)
    SZ_idx = jm.Placeholder("SZ_idx", ndim=1)  # [0..S-2]      (optional)

    # Pairs for M (m!=n) and MR (m<n), mirroring ZPL's "set M" and "set MR"
    M_n = jm.Placeholder("M_n")
    M_pair = jm.Placeholder("M_pair", ndim=2)  # shape (M_n, 2) rows <m,n> with m!=n
    MR_n = jm.Placeholder("MR_n")
    MR_pair = jm.Placeholder("MR_pair", ndim=2)  # shape (MR_n, 2) rows <m,n> with m<n

    # -------------------------
    # Decision variables (ZPL: var x[M*S], bh[T*S0], ba[T*S0])
    # -------------------------
    x = jm.BinaryVar(
        "x", shape=(T, S, T)
    )  # we'll index as x[m, s, n] to make "sum over n" & "sum over m" natural
    # it is equivalent to x[T,T,S]; just a dimension order choice.
    bh = jm.BinaryVar("bh", shape=(T, S))  # used only at s in S0_idx
    ba = jm.BinaryVar("ba", shape=(T, S))

    # -------------------------
    # Elements (loop indices)
    # -------------------------
    m = jm.Element("m", belong_to=T)
    n = jm.Element("n", belong_to=T)
    s = jm.Element("s", belong_to=S)
    ss0 = jm.Element("ss0", belong_to=S0)  # position in S0_idx
    q1 = jm.Element("q1", belong_to=S1)  # position in S1_idx
    rr = jm.Element("rr", belong_to=MR_n)  # row index in MR_pair
    mm = jm.Element("mm", belong_to=M_n)  # row index in M_pair
    t = jm.Element("t", belong_to=T)
    a = jm.Element("a", belong_to=T)  # sum over opponents a != t
    h = jm.Element("h", belong_to=T)  # sum over opponents h != t

    # Helper accessors (like dereferencing <m,n> in ZPL sets)
    def M_host(mm_):
        return M_pair[mm_, 0]

    def M_away(mm_):
        return M_pair[mm_, 1]

    def MR_left(rr_):
        return MR_pair[rr_, 0]

    def MR_right(rr_):
        return MR_pair[rr_, 1]

    # -------------------------
    # c1: For each ordered pair (m,n) in M, play exactly once at m's home
    #     ZPL: subto c1: forall <m,n> in M : sum <s> in S : x[m,n,s] == 1;
    # -------------------------
    P += jm.Constraint(
        "c1_each_pair_home_once",
        jm.sum(s, x[M_host(mm), s, M_away(mm)]) == 1,
        forall=[mm],
    )

    # -------------------------
    # c2: Per slot, exactly |T|/2 matches
    #     ZPL: subto c2: forall <s> in S: sum <m,n> in M : x[m,n,s] == matches_per_slot;
    # -------------------------
    P += jm.Constraint(
        "c2_matches_per_slot",
        jm.sum(mm, x[M_host(mm), s, M_away(mm)]) == matches_per_slot,
        forall=[s],
    )

    # -------------------------
    # c3: Per team & slot, exactly one game (home or away)
    #     ZPL: subto c3: forall <s> in S, forall <t> in T:
    #                    sum <t,n> in M : x[t,n,s] + sum <m,t> in M : x[m,t,s] == 1;
    #     We write it as sums over a!=t and h!=t.
    # -------------------------
    P += jm.Constraint(
        "c3_one_game_per_team_per_slot",
        jm.sum([(a, a != t)], x[t, s, a]) + jm.sum([(h, h != t)], x[h, s, t]) == 1,
        forall=[t, s],
    )

    # -------------------------
    # br_count: Break activation constraints (exactly as in your ZPL)
    #     ZPL:
    #       sum_a x[t,a,s-1] + sum_a x[t,a,s] - 1 <= bh[t,s]
    #       sum_h x[h,t,s-1] + sum_h x[h,t,s] - 1 <= ba[t,s]
    #     We pass s via S0_idx[ss0] and precompute s-1 via S0_prev[ss0].
    # -------------------------
    S0_prev = jm.Placeholder(
        "S0_prev", ndim=1
    )  # same length as S0_idx; S0_prev[pos] = S0_idx[pos] - 1

    P += jm.Constraint(
        "br_count_home",
        jm.sum([(a, a != t)], x[t, S0_prev[ss0], a])
        + jm.sum([(a, a != t)], x[t, S0_idx[ss0], a])
        - 1
        <= bh[t, S0_idx[ss0]],
        forall=[t, ss0],
    )

    P += jm.Constraint(
        "br_count_away",
        jm.sum([(h, h != t)], x[h, S0_prev[ss0], t])
        + jm.sum([(h, h != t)], x[h, S0_idx[ss0], t])
        - 1
        <= ba[t, S0_idx[ss0]],
        forall=[t, ss0],
    )

    # -------------------------
    # c4: Phased double round-robin (exactly one leg in first half S1)
    #     ZPL: subto c4: forall <m,n> in MR : sum <s> in S1 : (x[m,n,s] + x[n,m,s]) == 1;
    # -------------------------
    P += jm.Constraint(
        "c4_phased",
        jm.sum(
            q1,
            x[MR_left(rr), S1_idx[q1], MR_right(rr)]
            + x[MR_right(rr), S1_idx[q1], MR_left(rr)],
        )
        == 1,
        forall=[rr],
    )

    # -------------------------
    # ca4_*: Cartesian blocks with upper bounds
    #     ZPL example: sum <m,n,s> in A * B * W with m != n : x[m,n,s] <= U;
    #     We support many such lines as "blocks".
    # -------------------------
    B_ca = jm.Placeholder("B_ca")  # number of ca4 lines
    Amax = jm.Placeholder("Amax")  # max |A| across lines
    Bmax = jm.Placeholder("Bmax")  # max |B| across lines
    Wmax = jm.Placeholder("Wmax")  # max |W| across lines
    A_set = jm.Placeholder("A_set", ndim=2)  # (B_ca, Amax) team ids (pad if needed)
    B_set = jm.Placeholder("B_set", ndim=2)  # (B_ca, Bmax)
    W_set = jm.Placeholder("W_set", ndim=2)  # (B_ca, Wmax) slot ids
    A_len = jm.Placeholder("A_len", ndim=1)  # (B_ca,)
    B_len = jm.Placeholder("B_len", ndim=1)  # (B_ca,)
    W_len = jm.Placeholder("W_len", ndim=1)  # (B_ca,)
    U_cap = jm.Placeholder("U_cap", ndim=1)  # (B_ca,)

    b = jm.Element("b", belong_to=B_ca)
    ia = jm.Element("ia", belong_to=Amax)
    ib = jm.Element("ib", belong_to=Bmax)
    iw = jm.Element("iw", belong_to=Wmax)

    mA = A_set[b, ia]
    nB = B_set[b, ib]
    sW = W_set[b, iw]

    P += jm.Constraint(
        "ca4_block_caps",
        jm.sum(
            [(ia, ia < A_len[b])],
            jm.sum(
                [(ib, (ib < B_len[b]) & (nB != mA))],
                jm.sum([(iw, iw < W_len[b])], x[mA, sW, nB]),
            ),
        )
        <= U_cap[b],
        forall=[b],
    )

    # -------------------------
    # ga1_*: Upper and lower bounds over selected (host,away) pairs and slots
    #     ZPL has both <= and >= versions; we keep two lists to mirror them exactly.
    # -------------------------

    # Upper-bound list: sum_{(i,j) in P_u} sum_{s in W_u} x[i,s,j] <= U
    U_ga_n = jm.Placeholder("U_ga_n")
    U_pairsN = jm.Placeholder("U_pairsN")
    U_winsN = jm.Placeholder("U_winsN")
    U_pairs = jm.Placeholder("U_pairs", ndim=3)  # (U_ga_n, U_pairsN, 2)
    U_pair_len = jm.Placeholder("U_pair_len", ndim=1)
    U_wins = jm.Placeholder("U_wins", ndim=2)  # (U_ga_n, U_winsN)
    U_win_len = jm.Placeholder("U_win_len", ndim=1)
    U_bound = jm.Placeholder("U_bound", ndim=1)

    gu = jm.Element("gu", belong_to=U_ga_n)
    up = jm.Element("up", belong_to=U_pairsN)
    uw = jm.Element("uw", belong_to=U_winsN)

    ih_u = U_pairs[gu, up, 0]
    ja_u = U_pairs[gu, up, 1]
    sw_u = U_wins[gu, uw]

    P += jm.Constraint(
        "ga1_upper_blocks",
        jm.sum(
            [(uw, uw < U_win_len[gu])],
            jm.sum([(up, up < U_pair_len[gu])], x[ih_u, sw_u, ja_u]),
        )
        <= U_bound[gu],
        forall=[gu],
    )

    # Lower-bound list: sum_{(i,j) in P_l} sum_{s in W_l} x[i,s,j] >= L
    L_ga_n = jm.Placeholder("L_ga_n")
    L_pairsN = jm.Placeholder("L_pairsN")
    L_winsN = jm.Placeholder("L_winsN")
    L_pairs = jm.Placeholder("L_pairs", ndim=3)  # (L_ga_n, L_pairsN, 2)
    L_pair_len = jm.Placeholder("L_pair_len", ndim=1)
    L_wins = jm.Placeholder("L_wins", ndim=2)
    L_win_len = jm.Placeholder("L_win_len", ndim=1)
    L_bound = jm.Placeholder("L_bound", ndim=1)

    gl = jm.Element("gl", belong_to=L_ga_n)
    lp = jm.Element("lp", belong_to=L_pairsN)
    lw = jm.Element("lw", belong_to=L_winsN)

    ih_l = L_pairs[gl, lp, 0]
    ja_l = L_pairs[gl, lp, 1]
    sw_l = L_wins[gl, lw]

    P += jm.Constraint(
        "ga1_lower_blocks",
        jm.sum(
            [(lw, lw < L_win_len[gl])],
            jm.sum([(lp, lp < L_pair_len[gl])], x[ih_l, sw_l, ja_l]),
        )
        >= L_bound[gl],
        forall=[gl],
    )

    # -------------------------
    # br1_*: Per-team break caps over specified slot subsets
    #     ZPL: forall <t> in {..}: sum <s> in {...}: (bh[t,s] + ba[t,s]) <= UB;
    # -------------------------
    BR1_n = jm.Placeholder("BR1_n")
    BR1_wN = jm.Placeholder("BR1_wN")
    BR1_teams = jm.Placeholder(
        "BR1_teams", ndim=1
    )  # length BR1_n; each entry is a team id
    BR1_wins = jm.Placeholder("BR1_wins", ndim=2)  # (BR1_n, BR1_wN) slot ids
    BR1_wlen = jm.Placeholder("BR1_wlen", ndim=1)  # per-line actual length
    BR1_ub = jm.Placeholder("BR1_ub", ndim=1)  # per-line UB

    gb = jm.Element("gb", belong_to=BR1_n)
    bw = jm.Element("bw", belong_to=BR1_wN)
    tb = BR1_teams[gb]
    sw_b = BR1_wins[gb, bw]

    P += jm.Constraint(
        "br1_team_block_caps",
        jm.sum([(bw, bw < BR1_wlen[gb])], bh[tb, sw_b] + ba[tb, sw_b]) <= BR1_ub[gb],
        forall=[gb],
    )

    # -------------------------
    # br2_1: Global break cap
    #     ZPL: sum <t,s> in T * S0 : (bh[t,s] + ba[t,s]) <= 26;   (26 in your file)
    # -------------------------
    BR2_cap = jm.Placeholder("BR2_cap")
    P += jm.Constraint(
        "br2_global_cap",
        jm.sum(t, jm.sum(ss0, bh[t, S0_idx[ss0]] + ba[t, S0_idx[ss0]])) <= BR2_cap,
    )

    # -------------------------
    # Objective: feasibility (constant 0)
    # -------------------------
    P += jm.sum(t, 0)

    return P

In [2]:
problem = build_drr_model_exact()
problem

In [3]:
import numpy as np

# 基础参数
data = {}
data["T"]                   = 8
data["S"]                   = 14
data["S0"]                  = 13                 # slots 1..13
data["S1"]                  = 7                  # slots 0..6
data["S2"]                  = 7                  # slots 7..13
data["SZ"]                  = 13                 # slots 0..12
data["matches_per_slot"]   = data["T"] // 2     # =4

# 索引集合
data["T_idx"]              = np.arange(8)                  # [0,1,...,7]
data["S_idx"]              = np.arange(14)                 # [0,...,13]
data["S0_idx"]             = np.arange(1, 14)              # [1,...,13]
data["S0_prev"]            = np.arange(0, 13)              # s-1 for s in S0_idx
data["S1_idx"]             = np.arange(0, 7)               # [0..6]
data["S2_idx"]             = np.arange(7, 14)              # [7..13]
data["SZ_idx"]             = np.arange(0, 13)              # [0..12]

pairs_M = [(m, n) for m in range(8) for n in range(8) if m != n]
pairs_MR = [(m, n) for m in range(8) for n in range(m+1, 8)]

data["M_n"]      = len(pairs_M)  # 8*7 = 56
data["M_pair"]   = np.array(pairs_M, dtype=int)

data["MR_n"]     = len(pairs_MR)  # C(8,2)=28
data["MR_pair"]  = np.array(pairs_MR, dtype=int)

# ca4_2 example
# 主集合 = {0,4,6,7}, 客集合 = {0,4,6,7}, slots = {0,1,10,12}, upper bound = 2
data["B_ca"]   = 1
data["Amax"]   = 4
data["Bmax"]   = 4
data["Wmax"]   = 4

data["A_set"]  = np.array([[0,4,6,7]], dtype=int)
data["A_len"]  = np.array([4], dtype=int)
data["B_set"]  = np.array([[0,4,6,7]], dtype=int)
data["B_len"]  = np.array([4], dtype=int)
data["W_set"]  = np.array([[0,1,10,12]], dtype=int)
data["W_len"]  = np.array([4], dtype=int)
data["U_cap"]  = np.array([2], dtype=int)

# ga1_2 (lower bound = 2) example
# matches: (1,4), (4,3), (5,3) in slots {6,7,10}
data["L_ga_n"]     = 1
data["L_pairsN"]  = 3
data["L_winsN"]   = 3

data["L_pairs"]   = np.array([[(1,4), (4,3), (5,3)]], dtype=int)
data["L_pair_len"] = np.array([3], dtype=int)
data["L_wins"]    = np.array([[6,7,10]], dtype=int)
data["L_win_len"] = np.array([3], dtype=int)
data["L_bound"]   = np.array([2], dtype=int)

# no upper-bound GA in this example
data["U_ga_n"]     = 0
data["U_pairsN"]   = 0
data["U_winsN"]    = 0
data["U_pairs"]    = np.empty((0,0,2), dtype=int)
data["U_pair_len"] = np.array([], dtype=int)
data["U_wins"]     = np.empty((0,0), dtype=int)
data["U_win_len"]  = np.array([], dtype=int)
data["U_bound"]    = np.array([], dtype=int)

# 假设你有两条 br1 规则：team 0 in slots {4,5,6,8,10,12} upper 2; team 3 in slot {13} upper 0
data["BR1_n"]       = 2
data["BR1_wN"]      = 6
data["BR1_teams"]   = np.array([0, 3], dtype=int)
data["BR1_wins"]    = np.array([[4,5,6,8,10,12], [13, -1, -1, -1, -1, -1]], dtype=int)  # pad with -1
data["BR1_wlen"]    = np.array([6,1], dtype=int)
data["BR1_ub"]      = np.array([2, 0], dtype=int)

data["BR2_cap"]     = 26  # 全局上限


In [4]:
import ommx
ommx.v1.Instance = jm.Interpreter(data).eval_problem(problem)

In [5]:
ommx.v1.Instance.decision_variables_df

,kind,lower,upper,name,subscripts,description,substituted_value
id,,,,,,,
0,Binary,0.0,1.0,bh,"[0, 0]",<NA>,<NA>
1,Binary,0.0,1.0,bh,"[0, 1]",<NA>,<NA>
2,Binary,0.0,1.0,bh,"[0, 2]",<NA>,<NA>
3,Binary,0.0,1.0,bh,"[0, 3]",<NA>,<NA>
4,Binary,0.0,1.0,bh,"[0, 4]",<NA>,<NA>
...,...,...,...,...,...,...,...
1115,Binary,0.0,1.0,x,"[7, 13, 3]",<NA>,<NA>
1116,Binary,0.0,1.0,x,"[7, 13, 4]",<NA>,<NA>
1117,Binary,0.0,1.0,x,"[7, 13, 5]",<NA>,<NA>


In [6]:
import ommx_pyscipopt_adapter as scip_ad
solution = scip_ad.OMMXPySCIPOptAdapter.solve(ommx.v1.Instance)
print(f"{solution.objective=}, {solution.feasible=}")

solution.objective=0.0, solution.feasible=True


In [7]:
solution.decision_variables_df

,kind,lower,upper,name,subscripts,description,substituted_value,value
id,,,,,,,,
0,Binary,0.0,1.0,bh,"[0, 0]",<NA>,<NA>,0.0
1,Binary,0.0,1.0,bh,"[0, 1]",<NA>,<NA>,0.0
2,Binary,0.0,1.0,bh,"[0, 2]",<NA>,<NA>,0.0
3,Binary,0.0,1.0,bh,"[0, 3]",<NA>,<NA>,0.0
4,Binary,0.0,1.0,bh,"[0, 4]",<NA>,<NA>,0.0
...,...,...,...,...,...,...,...,...
1115,Binary,0.0,1.0,x,"[7, 13, 3]",<NA>,<NA>,0.0
1116,Binary,0.0,1.0,x,"[7, 13, 4]",<NA>,<NA>,0.0
1117,Binary,0.0,1.0,x,"[7, 13, 5]",<NA>,<NA>,0.0
